#Universidade Federal do Rio Grande do Norte
####DIM0438 - REDES DE COMPUTADORES - T02 (2022.1)
####Classificação de tweets com spaCy

O objetivo desse trabalho é coletar tweets utilizando API do twitter e posteriormente classificalos.

# Etapa 01: Conectando API do Twitter e extraindo Tweets com Python 

In [ ]:
import tweepy as tw


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
with open('/content/gdrive/MyDrive/Trabalho REDES/token.txt', 'r') as tfile:
    consumer_key = tfile.readline().strip('\n')
    consumer_secret = tfile.readline().strip('\n')
    access_token = tfile.readline().strip('\n')
    access_token_secret = tfile.readline().strip('\n')

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tw.API(auth)

In [ ]:
meus_tweets = api.home_timeline()

for tweet in meus_tweets:
    print(tweet.text)

#ufrn #laireufrn https://t.co/qSR8rB4aCe
Segue o 🧶👇🏼 https://t.co/TnoBLcIXwx
Você sabia que existe um tipo de hipertensão que atinge o pulmão? 🫁

O Laire (Laboratório de Avaliação e Intervençã… https://t.co/jrquWfBDlK
👏🏼 O @mccufrn registrou sábado recorde de visitação, no encerramento da 20ª Semana Nacional de Museus.

Contamos so… https://t.co/e83z6OAamY
Universidades Federais sofrem bloqueio orçamentário 👉🏼 https://t.co/1S5Rls5I87

#ufrn
RT @ISDnarede: Atenção graduados das áreas de biológicas, engenharias e saúde!

Estão abertas as inscrições para o Mestrado em Neuroengenha…
Estivemos renovando a nossa esperança em dias melhores, na Conferência de Tática Eleitoral do @13PTRN. Juntos pela… https://t.co/AiyNvprDXe
Em nome do povo do RN, toda nossa solidariedade nesse momento de dor que o povo pernambucano vive. O nosso Estado e… https://t.co/AjTOgMftGu
Contactei o governador @PauloCamara40 e desde ontem autorizei o envio de 8 bombeiros (5 especializados em buscas e… https://t.co/FmYD

In [ ]:
pesquisa = api.home_timeline()

resultado = []

for twitters in pesquisa:
  resultado.append(twitters.text)
  


In [ ]:
print(resultado[0])

Universidades Federais sofrem bloqueio orçamentário 👉🏼 https://t.co/1S5Rls5I87

#ufrn


# Treinando o modelo de classificação de textos com spaCy

Importação e instalação das bibliotecas

In [ ]:
!pip install -q spacy==2.2.3

     |████████████████████████████████| 10.4 MB 5.1 MB/s 
     |████████████████████████████████| 2.2 MB 48.4 MB/s 


In [ ]:
import spacy
spacy.__version__

'2.2.3'

In [ ]:
!python3 -m spacy download pt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.2 MB 1.0 MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-py3-none-any.whl size=21186281 sha256=a9dbfe7a2fa2d144097c78d950a8cae309e98913e737edbb24c7a0ea4545ecc4
  Stored in directory: /tmp/pip-ephem-wheel-cache-mxq9a1om/wheels/c3/f9/0c/5c014a36941a00f5df5fc0756cb961d7c457a978e697a6ce3b
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [ ]:
import pandas as pd
import string
import spacy
import random
import seaborn as sns
import numpy as np
import re

Carregamento das bases de dados

Base de treinamento

Negative label: 0

Positive label: 1

In [ ]:
base_treinamento = pd.read_csv('/content/gdrive/MyDrive/Trabalho REDES/Train50.csv', delimiter=';')

In [ ]:
base_treinamento.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1050785521201541121,@Laranjito76 A pessoa certa para isso seria o ...,Fri Oct 12 16:29:25 +0000 2018,1,:)
1,1050785431955140608,"@behin_d_curtain Para mim, é precisamente o co...",Fri Oct 12 16:29:04 +0000 2018,1,:)
2,1050785401248645120,Vou fazer um video hoje... estou pensando em f...,Fri Oct 12 16:28:56 +0000 2018,1,:)
3,1050785370982547461,"aaaaaaaa amei tanto essas polaroids, nem sei e...",Fri Oct 12 16:28:49 +0000 2018,1,:)
4,1050785368902131713,Valoriza o coração do menininho que vc tem. El...,Fri Oct 12 16:28:49 +0000 2018,1,:)


In [ ]:
base_treinamento.drop(['id', 'tweet_date', 'query_used'], axis = 1, inplace=True)

In [ ]:
base_treinamento.head()

,tweet_text,sentiment
0,@Laranjito76 A pessoa certa para isso seria o ...,1
1,"@behin_d_curtain Para mim, é precisamente o co...",1
2,Vou fazer um video hoje... estou pensando em f...,1
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1
4,Valoriza o coração do menininho que vc tem. El...,1


Base de teste

In [ ]:
base_teste = pd.read_csv('/content/gdrive/MyDrive/Trabalho REDES/Test.csv', delimiter=';')

In [ ]:
base_teste.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1029536486021099522,@Gazo1a Nossa! Muito obrigada :),Wed Aug 15 01:13:20 +0000 2018,1,:)
1,1029536496368406528,@BerzGamer vai pa puta que te pariu :),Wed Aug 15 01:13:23 +0000 2018,1,:)
2,1029536531655131137,QUER MAIS DESCONTOS? (14/08) ⭐⭐⭐⭐⭐ 🌐 Confira n...,Wed Aug 15 01:13:31 +0000 2018,1,:)
3,1029536560117678081,"EU VOU PEGAR VCS, ME AJUDEM GALERA, PELO AMOR ...",Wed Aug 15 01:13:38 +0000 2018,1,:)
4,1029536605852377088,Estávamos em casa do Zé e eu estava a morrer d...,Wed Aug 15 01:13:49 +0000 2018,1,:)


In [ ]:
base_teste.drop(['id', 'tweet_date', 'query_used'], axis = 1, inplace=True)

In [ ]:
base_teste.head()

,tweet_text,sentiment
0,@Gazo1a Nossa! Muito obrigada :),1
1,@BerzGamer vai pa puta que te pariu :),1
2,QUER MAIS DESCONTOS? (14/08) ⭐⭐⭐⭐⭐ 🌐 Confira n...,1
3,"EU VOU PEGAR VCS, ME AJUDEM GALERA, PELO AMOR ...",1
4,Estávamos em casa do Zé e eu estava a morrer d...,1


Função para pré-processamento dos textos

- Letras minúsculas
- Nome do usuário (@)
- URLs
- Espaços em branco
- Emoticons
- Stop words
- Lematização
- Pontuações

In [ ]:
pln = spacy.load('pt')
pln

In [ ]:
base_treinamento['tweet_text'][1]

'@behin_d_curtain Para mim, é precisamente o contrário :) Vem a chuva e vem a boa disposição :)'

In [ ]:
stop_words = spacy.lang.pt.stop_words.STOP_WORDS

In [ ]:
print(stop_words)

{'ora', 'vinda', 'era', 'quieto', 'sobre', 'estiveram', 'mal', 'tivemos', 'tuas', 'usar', 'tente', 'estes', 'tendes', 'apoio', 'eventual', 'nós', 'este', 'fostes', 'quer', 'num', 'sem', 'umas', 'novos', 'puderam', 'des', 'ponto', 'outras', 'fará', 'relação', 'são', 'geral', 'vai', 'primeira', 'fomos', 'falta', 'isso', 'nuns', 'meio', 'dar', 'bastante', 'terceira', 'deste', 'último', 'tempo', 'tarde', 'mesmo', 'depois', 'está', 'vez', 'nesta', 'vos', 'naquele', 'esse', 'só', 'dentro', 'vocês', 'teus', 'sétima', 'pegar', 'uma', 'aqui', 'cá', 'aquelas', 'conhecida', 'tiveram', 'à', 'pôde', 'numa', 'grandes', 'baixo', 'logo', 'quanto', 'ir', 'inclusive', 'não', 'podia', 'que', 'de', 'maioria', 'momento', 'ainda', 'quatro', 'possível', 'do', 'cinco', 'põem', 'quem', 'sexto', 'possivelmente', 'ela', 'tão', 'vêm', 'próprio', 'porquanto', 'porquê', 'menor', 'próxima', 'sétimo', 'dá', 'tanto', 'adeus', 'aos', 'tentar', 'têm', 'porém', 'dois', 'teve', 'aquele', 'estado', 'estive', 'enquanto', 'e

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def preprocessamento(texto):
  # Letras minúsculas
  texto = texto.lower()

  # Nome do usuário
  texto = re.sub(r"@[A-Za-z0-9$-_@.&+]+", ' ', texto)

  # URLs
  texto = re.sub(r"https?://[A-Za-z0-9./]+", ' ', texto)

  # Espaços em branco
  texto = re.sub(r" +", ' ', texto)

  # Emoticons
  lista_emocoes = {':)': 'emocaopositiva',
                   ':d': 'emocaopositiva',
                   ':(': 'emocaonegativa'}
  for emocao in lista_emocoes:
    texto = texto.replace(emocao, lista_emocoes[emocao])

  # Lematização
  documento = pln(texto)

  lista = []
  for token in documento:
    lista.append(token.lemma_)
  
  # Stop words e pontuações
  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in string.punctuation]
  lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])
  
  return lista

In [ ]:
texto_teste = '@behin_d_curtain :D Para :( mim, http://www.iaexpert.com.br é precisamente o contrário :) Vem a chuva e vem a boa disposição :)'
resultado = preprocessamento(texto_teste)
resultado

'  emocaopositiva parir emocaonegativa mim precisamente o contrário emocaopositiva vir o chuva e vir o disposição emocaopositiva'

Pré-processamento da base de dados




In [ ]:
base_treinamento.head(10)

,tweet_text,sentiment
0,@Laranjito76 A pessoa certa para isso seria o ...,1
1,"@behin_d_curtain Para mim, é precisamente o co...",1
2,Vou fazer um video hoje... estou pensando em f...,1
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1
4,Valoriza o coração do menininho que vc tem. El...,1
5,@KingJokerLeto mas amiga eu to aqui ainda :),1
6,"@RivasJairo Bravo, Jairo!!! :)))",1
7,@inquilinasls @Spotify socorro ksoaksosks acho...,1
8,"@chimekarlla gosto muito de fazenda :) , mas m...",1
9,Amanha vou ver venom :D,1


In [ ]:
base_treinamento['tweet_text'] = base_treinamento['tweet_text'].apply(preprocessamento)

In [ ]:
base_treinamento.head(10)

,tweet_text,sentiment
0,o pessoa certo parir seriar o valer e azeved...,1
1,parir mim precisamente o contrário emocaopos...,1
2,video hoje ... pensar falar sobrar o meter csg...,1
3,aaaaaaaa amar polaroids expressar o quantum to...,1
4,valorizar o coração menino vc diferente o sorr...,1
5,amigo to emocaopositiva,1
6,bravo jairo emocaopositiva,1
7,socorrer ksoaksosks achar to o caixa som :p,1
8,gostar fazenda emocaopositiva morrer medo ga...,1
9,amanhar venom emocaopositiva,1


In [ ]:
base_teste['tweet_text'] = base_teste['tweet_text'].apply(preprocessamento)

In [ ]:
base_teste.head(10)

,tweet_text,sentiment
0,obrigar emocaopositiva,1
1,pa puta parir emocaopositiva,1
2,querer desconto 14/08 ⭐ ⭐ ⭐ ⭐ ⭐ 🌐 conferir o l...,1
3,vcs ajudar galera pelar amor butera emocaoposi...,1
4,casar zé e o morrer sono chegar o casar e fica...,1
5,preciso emocaopositiva,1
6,acaso adorar beijo e abraço hahahahaha bue f...,1
7,solteiro s emocaopositiva,1
8,lindeza cortador kit mário coleção bia cravol ...,1
9,claro emocaopositiva,1


Tratamento da classe

In [ ]:
exemplo_base_dados = [["este trabalho é agradável", {"POSITIVO": True, "NEGATIVO": False}],
                      ["este lugar continua assustador", {"POSITIVO": False, "NEGATIVO": True}]]

In [ ]:
base_dados_treinamento_final = []
for texto, emocao in zip(base_treinamento['tweet_text'], base_treinamento['sentiment']):
  if emocao == 1:
    dic = ({'POSITIVO': True, 'NEGATIVO': False})
  elif emocao == 0:
    dic = ({'POSITIVO': False, 'NEGATIVO': True})

  base_dados_treinamento_final.append([texto, dic.copy()])

In [ ]:
base_dados_treinamento_final[10:15]

[['simplesmente o deixar chatear querer muitooo e iludir emocaopositiva',
  {'NEGATIVO': False, 'POSITIVO': True}],
 ['o comedir umar rotina e perfeitamente parir consolidar o escola condução :p',
  {'NEGATIVO': False, 'POSITIVO': True}],
 ['amigar parecer apaixonar vc — recíproco emocaopositiva',
  {'NEGATIVO': False, 'POSITIVO': True}],
 ['  gajar bacano aleijar esperar e o silvar feliz emocaopositiva',
  {'NEGATIVO': False, 'POSITIVO': True}],
 ['tou o video cabra bebé emocaopositiva',
  {'NEGATIVO': False, 'POSITIVO': True}]]

In [ ]:
base_dados_treinamento_final[45000:45005]

[['o menino pijama listrar o filmar triste mundo estruturar nenhum pra assistir rever e to pensar n emocaonegativa',
  {'NEGATIVO': True, 'POSITIVO': False}],
 ['pra o vídeo luísa raiz o poliana infelizmente o twitter tá ruim amanhã postar emocaonegativa asaventurasdepoliana099',
  {'NEGATIVO': True, 'POSITIVO': False}],
 ['to crise abstinência strangers emocaonegativa — tbm :/ to',
  {'NEGATIVO': True, 'POSITIVO': False}],
 ['  esquecer olhar pra pagar o mensalidade turma pro cursar querer emocaonegativa',
  {'NEGATIVO': True, 'POSITIVO': False}],
 ['querer comer doce comer montar hoje emocaonegativa maldição',
  {'NEGATIVO': True, 'POSITIVO': False}]]

Criação do classificador

In [ ]:
modelo = spacy.blank('pt')
categorias = modelo.create_pipe("textcat")
categorias.add_label("POSITIVO")
categorias.add_label("NEGATIVO")
modelo.add_pipe(categorias)
historico = []

In [ ]:
modelo.begin_training()
for epoca in range(20):
  random.shuffle(base_dados_treinamento_final)
  losses = {}
  for batch in spacy.util.minibatch(base_dados_treinamento_final, 512):
    textos = [modelo(texto) for texto, entities in batch]
    annotations = [{'cats': entities} for texto, entities in batch]
    modelo.update(textos, annotations, losses=losses)
    historico.append(losses)
  if epoca % 5 == 0:
    print(losses)

{'textcat': 3.0378386168816718e-05}
{'textcat': 1.4181678481030302e-07}
{'textcat': 1.4284329142872504e-07}
{'textcat': 3.132817156074885e-07}


In [ ]:
historico_loss = []
for i in historico:
  historico_loss.append(i.get('textcat'))

In [ ]:
modelo.to_disk("modelo")

#  Testando o modelo de classificação com tweets coletado em tempo real com API.

In [ ]:
modelo_carregado = spacy.load('modelo')
modelo_carregado

In [ ]:
meus_tweets = api.home_timeline()

for tweet in meus_tweets:
    print(tweet.text)

hoje estou muito feliz pois vou ter aula de redes
Estou muito triste com a prova de redes de computadores hoje
#ufrn #laireufrn https://t.co/qSR8rB4aCe
Segue o 🧶👇🏼 https://t.co/TnoBLcIXwx
Você sabia que existe um tipo de hipertensão que atinge o pulmão? 🫁

O Laire (Laboratório de Avaliação e Intervençã… https://t.co/jrquWfBDlK
👏🏼 O @mccufrn registrou sábado recorde de visitação, no encerramento da 20ª Semana Nacional de Museus.

Contamos so… https://t.co/e83z6OAamY
Universidades Federais sofrem bloqueio orçamentário 👉🏼 https://t.co/1S5Rls5I87

#ufrn
RT @ISDnarede: Atenção graduados das áreas de biológicas, engenharias e saúde!

Estão abertas as inscrições para o Mestrado em Neuroengenha…
Estivemos renovando a nossa esperança em dias melhores, na Conferência de Tática Eleitoral do @13PTRN. Juntos pela… https://t.co/AiyNvprDXe
Em nome do povo do RN, toda nossa solidariedade nesse momento de dor que o povo pernambucano vive. O nosso Estado e… https://t.co/AjTOgMftGu
Contactei o governador

In [ ]:
pesquisa = api.home_timeline()

resultado = []

for twitters in pesquisa:
  resultado.append(twitters.text)

In [ ]:
print(resultado[0])

hoje estou muito feliz pois vou ter aula de redes


In [ ]:
print(resultado[1])

Estou muito triste com a prova de redes de computadores hoje


In [ ]:
modelo_carregado(resultado[0]).cats

{'NEGATIVO': 0.0010585126001387835, 'POSITIVO': 0.9945263266563416}

In [ ]:
modelo_carregado(resultado[1]).cats

{'NEGATIVO': 0.9808638095855713, 'POSITIVO': 0.013057216070592403}